In [29]:
# 读取mat文件
import scipy.io as sio
import numpy as np

# 读取mat文件
data = sio.loadmat('./VehicleInfo.mat')

# 获取数据
print(data.keys())

info = data['VehicleInfo']

print(info)

dict_keys(['__header__', '__version__', '__globals__', 'VehicleInfo'])
[[(array(['vehicle_0000001.jpg'], dtype='<U19'), array([[1080]], dtype=uint16), array([[1920]], dtype=uint16), array([[(array([[597]], dtype=uint16), array([[3]], dtype=uint8), array([[935]], dtype=uint16), array([[341]], dtype=uint16), array(['SUV'], dtype='<U3'))]],
        dtype=[('left', 'O'), ('top', 'O'), ('right', 'O'), ('bottom', 'O'), ('category', 'O')]), array([[1]], dtype=uint8))                                                                                                                                                ]
 [(array(['vehicle_0000002.jpg'], dtype='<U19'), array([[1080]], dtype=uint16), array([[1920]], dtype=uint16), array([[(array([[399]], dtype=uint16), array([[283]], dtype=uint16), array([[875]], dtype=uint16), array([[874]], dtype=uint16), array(['SUV'], dtype='<U3'))]],
        dtype=[('left', 'O'), ('top', 'O'), ('right', 'O'), ('bottom', 'O'), ('category', 'O')]), array([[1]], dtype=u

## 制作文件名和比例坐标

In [55]:
names = info['name']

img_height = info['height']
img_width = info['width']

loc = info['vehicles']
nums = info['nVehicles']

In [56]:
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0    # (x_min + x_max) / 2.0
    y = (box[2] + box[3]) / 2.0    # (y_min + y_max) / 2.0
    w = box[1] - box[0]   # x_max - x_min
    h = box[3] - box[2]   # y_max - y_min
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

In [92]:
dict = {'SUV':0 , 'Sedan':1, 'Microbus':2, 'Minivan':3, 'Truck':4, 'Bus':5}

In [95]:
for i in range(names.shape[0]):
    name = names[i][0][0]
    print(name)

    n = nums[i][0][0][0]

    for j in range(n):
        x1, y1, x2, y2 = loc[i][0][0][0][0][0][0], loc[i][0][0][0][1][0][0], loc[i][0][0][0][2][0][0], loc[i][0][0][0][3][0][0]
        height, width = img_height[0][0][0][0], img_width[0][0][0][0]

        x,y,w,h = convert((height, width), (x1, x2, y1, y2))

        print(x, y, w, h)

        label = loc[i][0][0][0][4][0]
        id = dict[label]

        print(id)

vehicle_0000001.jpg
0.7092592592592593 0.08958333333333333 0.312962962962963 0.17604166666666665
0
vehicle_0000002.jpg
0.5898148148148148 0.3013020833333333 0.44074074074074077 0.3078125
0
vehicle_0000003.jpg
0.725462962962963 0.16536458333333334 0.3453703703703704 0.2296875
0
vehicle_0000004.jpg
0.763425925925926 0.08151041666666667 0.28425925925925927 0.1578125
1
vehicle_0000005.jpg
0.6615740740740741 0.30859375 0.39351851851851855 0.26197916666666665
1
vehicle_0000006.jpg
1.1462962962962964 0.2015625 0.34444444444444444 0.221875
1
vehicle_0000007.jpg
1.1574074074074074 0.3346354166666667 0.4185185185185185 0.2890625
1
vehicle_0000008.jpg
1.1587962962962963 0.15807291666666667 0.3138888888888889 0.1953125
1
vehicle_0000009.jpg
1.1791666666666667 0.3390625 0.41574074074074074 0.28229166666666666
1
1.1791666666666667 0.3390625 0.41574074074074074 0.28229166666666666
1
vehicle_0000010.jpg
0.7055555555555556 0.08046875 0.2537037037037037 0.15989583333333332
2
vehicle_0000011.jpg
0.563425

: 

In [86]:
import os
import cv2
import matplotlib.pyplot as plt

image_path = '/home/ubuntu/下载/dataset/'
goal_path_test = '/home/ubuntu/下载/dataset/archive/test_label'
goal_path_train = '/home/ubuntu/下载/dataset/archive/train_label'
img_path_test = '/home/ubuntu/下载/dataset/archive/img_test'
img_path_train = '/home/ubuntu/下载/dataset/archive/img_train'

# 设置一个set，每个label只绘制一次
set1 = []

# 遍历文件夹
for i in range(annotations.shape[1]):
    name = annotations[0][i][0][0]
    x1, y1, x2, y2 = annotations[0][i][1][0], annotations[0][i][2][0], annotations[0][i][3][0], annotations[0][i][4][0]
    classes = annotations[0][i][5][0][0] - 1
    te_tr = annotations[0][i][6][0][0]

    # first = label_dict[0][classes][0][0]

    file = os.path.join(image_path, name)
    # print(file)

    # 读取图片
    img = cv2.imread(file)

    # 获取图片的高和宽
    height, width = img.shape[0], img.shape[1]

    # 得到yolo的百分比位置标注
    x, y, w, h = convert((width, height), (x1, x2, y1, y2))

    # 从原始字典中找到标签，再映射到UA-Det的标签
    label = label_dict[0][classes][0]
    label = dict[label]

    str = f'{label} {x[0]} {y[0]} {w[0]} {h[0]}'
    print(str)

    if classes not in set1:
        print(label_dict[0][classes][0] , '---> ', label)

        # 绘制框
        # cv2.rectangle(img, (x1[0], y1[0]), (x2[0], y2[0]), (0, 255, 0), 2)

        plt.imshow(img)
        plt.show()
        set1.append(classes)

    file_name = name.split('/')[1].split('.')[0]
    if te_tr == 1:
        # 保存图片
        img_name = os.path.join(img_path_test, file_name + '.jpg')
        cv2.imwrite(img_name, img)

        txt_name = os.path.join(goal_path_test, file_name + '.txt')
        with open(txt_name, 'w') as f:
            f.write(str)

    elif te_tr == 0:
        # 保存图片
        img_name = os.path.join(img_path_train, file_name + '.jpg')
        cv2.imwrite(img_name, img)

        txt_name = os.path.join(goal_path_train, file_name + '.txt')
        with open(txt_name, 'w') as f:
            f.write(str)

(9850, 1)
